In [2]:
import os
os.chdir('..')

In [2]:
pwd

'/mnt/md1/user_victor/automatic_melody_harmonization'

In [3]:
from tonal import pianoroll2number, joint_prob2pianoroll96
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from pypianoroll import Multitrack, Track
import pypianoroll as pr
import pickle
from matplotlib import pyplot as plt
import os
import random
from model.Parameter_CVAE import *
from decode import *

In [38]:
from tonal import pianoroll2number, joint_prob2pianoroll96
import numpy as np
from pypianoroll import Multitrack, Track
import pypianoroll as pr
from matplotlib import pyplot as plt
import os 

# Append argmax index to get pianoroll array
#[batch, beats = 272, chordtypes = 96]
def argmax2pianoroll(joint_prob):
    chord_pianoroll = []
    for song in joint_prob:
        pianoroll = []
        for beat in song:
            pianoroll.append(joint_prob2pianoroll96(beat))
        chord_pianoroll.append(pianoroll)

    chord_pianoroll = np.asarray(chord_pianoroll)

    accompany_pianoroll = chord_pianoroll * 100
    print(chord_pianoroll.shape)
    return accompany_pianoroll

# augment chord into frame base
def sequence2frame(accompany_pianoroll, chord_groundtruth, BEAT_RESOLUTION=24, BEAT_PER_CHORD=2):
    print('augment chord into frame base...')
    accompany_pianoroll_frame = []
    chord_groundtruth_frame = []
    for acc_song, truth_song in zip(accompany_pianoroll, chord_groundtruth):
        acc_pianoroll = []
        truth_pianoroll = []
        for acc_beat, truth_beat in zip(acc_song, truth_song):
            for i in range(BEAT_RESOLUTION*BEAT_PER_CHORD):
                acc_pianoroll.append(acc_beat)
                truth_pianoroll.append(truth_beat)
        accompany_pianoroll_frame.append(acc_pianoroll)
        chord_groundtruth_frame.append(truth_pianoroll)

    accompany_pianoroll_frame = np.asarray(accompany_pianoroll_frame).astype(int)
    chord_groundtruth_frame = np.asarray(chord_groundtruth_frame)
    print('accompany_pianoroll shape:', accompany_pianoroll_frame.shape)
    print('groundtruth_pianoroll shape:', chord_groundtruth_frame.shape)
    return accompany_pianoroll_frame, chord_groundtruth_frame

# write pianoroll
def write_pianoroll(result_dir, melody_data, accompany_pianoroll_frame,chord_groundtruth_frame, length, tempos,downbeats, BEAT_RESOLUTION=24, BEAT_PER_CHORD=2):

    print('write pianoroll...')
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    counter = 0
    for melody_roll, chord_roll, truth_roll, l, tempo, downbeat in zip(melody_data, accompany_pianoroll_frame,
                                                                            chord_groundtruth_frame, length, tempos,
                                                                            downbeats):
        
        melody_roll, chord_roll, truth_roll = melody_roll[:l], chord_roll[:l], truth_roll[:l]

        print(l)
        print(melody_roll.shape, chord_roll.shape, truth_roll.shape)
        print(tempo, downbeat)
        
        track1 = Track(pianoroll=melody_roll)
        track2 = Track(pianoroll=chord_roll)
        track3 = Track(pianoroll=truth_roll)

        generate = Multitrack(tracks=[track1, track2], tempo=tempo, downbeat=downbeat, BEAT_RESOLUTION=BEAT_RESOLUTION)
        truth = Multitrack(tracks=[track1, track3], tempo=tempo, downbeat=downbeat, BEAT_RESOLUTION=BEAT_RESOLUTION)

        pr.write(generate, result_dir + '/generate_' + str(counter) + '.mid')
        pr.write(truth, result_dir + '/groundtruth_' + str(counter) + '.mid')

        fig, axs = generate.plot()
        plt.savefig(result_dir + '/generate_' + str(counter) + '.png')
        plt.close()
        fig, axs = truth.plot()
        plt.savefig(result_dir + '/groundtruth_' + str(counter) + '.png')
        plt.close()

        counter += 1
    
    print('Finished!')
    

# write one pianoroll at once
def write_one_pianoroll(result_dir, filename, melody_data, accompany_pianoroll_frame,chord_groundtruth_frame, length, tempo ,downbeat, BEAT_RESOLUTION=24, BEAT_PER_CHORD=2):

    print('write pianoroll...')
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
        
    l = length

    melody_roll, chord_roll, truth_roll = melody_data[0][:l], accompany_pianoroll_frame[0][:l], chord_groundtruth_frame[0][:l]

    print(l)
    print(melody_roll.shape, chord_roll.shape, truth_roll.shape)
    print(tempo, downbeat)
    track1 = Track(pianoroll=melody_roll)
    track2 = Track(pianoroll=chord_roll)
    track3 = Track(pianoroll=truth_roll)

    generate = Multitrack(tracks=[track1, track2], tempo=tempo[0], downbeat=downbeat[0], BEAT_RESOLUTION=BEAT_RESOLUTION)
    truth = Multitrack(tracks=[track1, track3], tempo=tempo[0], downbeat=downbeat[0], BEAT_RESOLUTION=BEAT_RESOLUTION)

    pr.write(generate, result_dir + '/generate-' + filename + '.mid')
    pr.write(truth, result_dir + '/groundtruth-' + filename + '.mid')

    fig, axs = generate.plot()
    plt.savefig(result_dir + '/generate-' + filename + '.png')
    plt.close()
    fig, axs = truth.plot()
    plt.savefig(result_dir + '/groundtruth-' + filename + '.png')
    plt.close()

    print('Finished!')

In [5]:
from metrics import CHE_and_CC, CTD, CTnCTR, PCS, MCTD

In [4]:
# Load data
device = 'cpu'
val_size = 500

print('loading data...')
melody_data = np.load('./melody_data.npy')
chord_groundtruth = np.load('./chord_groundtruth.npy')
chord_onehot = np.load('./onehot_96.npy')

# Reconstructed chords
# chord_recon = np.load('./reconstructed_one_hot_chords.npy')

melody = np.load('./melody_baseline.npy')
lengths = np.load('./length.npy')

f = open('tempos', 'rb')
tempos = pickle.load(f)
f.close()
f = open('downbeats', 'rb')
downbeats = pickle.load(f)
f.close()

print('splitting testing set...')
melody_data = melody_data[:val_size]
chord_groundtruth = chord_groundtruth[:val_size]

val_chord = torch.from_numpy(chord_onehot[:val_size]).float()
val_melody = torch.from_numpy(melody[:val_size]).float()
val_length = torch.from_numpy(lengths[:val_size])

tempos = tempos[:val_size]
downbeats = downbeats[:val_size]

loading data...
splitting testing set...


In [5]:
val_length.shape

torch.Size([500])

In [7]:
val_length[0].item()

17

In [ ]:
print(val_melody.shape)
print(melody_data.shape)

In [5]:
# Load model
print('building model...')
model = CVAE(device = device).to(device)
model.load_state_dict(torch.load('output_models/model_parameter_cvae_weighting.pth'))

model.eval()
val_length = val_length.to(device)

building model...


In [9]:
# chord_groundtruth.shape
val_length.shape

torch.Size([500])

In [6]:
latent_size = 16
pianoroll_frames = 12 * 24 * 2
index = 10

melody_truth = np.expand_dims(melody_data[index], axis=0)
chord_truth = np.expand_dims(chord_groundtruth[index], axis=0)
tempo = [tempos[index]]
downbeat = [downbeats[index]]

melody1 = val_melody[index].unsqueeze(dim=0)
length1 = torch.Tensor([val_length[index]]).long()


In [11]:
length1

tensor([32])

In [34]:
# Sampling
seed = 30
torch.manual_seed(seed)

batch_size = 1
pitch = 1
rhythm = 0

r_pitch = torch.Tensor([pitch])
r_rhythm = torch.Tensor([rhythm])

r_pitch = r_pitch.view(batch_size,1,1).expand(batch_size,272,1).to(device)
r_rhythm = r_rhythm.view(batch_size,1,1).expand(batch_size,272,1).to(device)

latent = torch.rand(1,272,latent_size)

z = torch.cat((latent,melody1,r_pitch,r_rhythm), dim=-1)

_, chord_pred = model.decode(z,length1)

In [8]:
chord_pred

tensor([[[9.9415e-04, 9.6845e-01, 2.9879e-23,  ..., 3.2372e-23,
          9.3268e-24, 6.5183e-24],
         [7.2570e-04, 5.7822e-01, 2.9097e-22,  ..., 3.3974e-22,
          1.1859e-22, 8.0768e-23],
         [1.9076e-02, 4.6440e-01, 2.5253e-21,  ..., 4.1953e-21,
          2.6621e-21, 1.2618e-21],
         ...,
         [9.9258e-01, 4.9705e-03, 1.6767e-06,  ..., 1.0256e-06,
          1.6412e-06, 1.2689e-06],
         [9.9209e-01, 5.7182e-03, 1.1110e-06,  ..., 6.7392e-07,
          1.0634e-06, 7.8767e-07],
         [9.9387e-01, 4.3889e-03, 1.1856e-06,  ..., 7.5921e-07,
          1.2047e-06, 9.1897e-07]]], grad_fn=<SoftmaxBackward>)

In [42]:
melody_truth.shape

(1, 13056, 128)

In [41]:
chord_pred.shape

torch.Size([1, 272, 96])

In [86]:
downbeat.shape

(768,)

In [39]:
########## Random sampling ###########
# Proceed chord decode
print('proceed chord decode...')
decode_length = length1
joint_prob = chord_pred.cpu().detach().numpy()

# Append argmax index to get pianoroll array
accompany_pianoroll = argmax2pianoroll(joint_prob)

# augment chord into frame base
BEAT_RESOLUTION = 24
BEAT_PER_CHORD = 2

accompany_pianoroll_frame, chord_groundtruth_frame = sequence2frame(accompany_pianoroll, chord_truth, BEAT_RESOLUTION=BEAT_RESOLUTION, BEAT_PER_CHORD=BEAT_PER_CHORD)

# length into frame base
decode_length = decode_length * BEAT_RESOLUTION * BEAT_PER_CHORD

# write pianoroll
result_dir = 'results/parameter_cvae_sample_result'
filename = str(index) + '-pitch-' + str(pitch) + '-rhythm-' + str(rhythm)
write_one_pianoroll(result_dir, filename ,melody_truth, accompany_pianoroll_frame,chord_groundtruth_frame, decode_length, tempo,downbeat)
# write_pianoroll(result_dir,melody_truth, accompany_pianoroll_frame,chord_groundtruth_frame, decode_length, tempo,downbeat)


proceed chord decode...
(1, 272, 128)
augment chord into frame base...
accompany_pianoroll shape: (1, 13056, 128)
groundtruth_pianoroll shape: (1, 13056, 128)
write pianoroll...
tensor([1536])
(1536, 128) (1536, 128) (1536, 128)
[array([130.00013, 130.00013, 130.00013, ..., 130.00013, 130.00013,
       130.00013])] [array([ True, False, False, ..., False, False, False])]
Finished!


In [18]:
write_pianoroll(result_dir,melody_truth, accompany_pianoroll_frame,chord_groundtruth_frame, decode_length, tempo,downbeat)


write pianoroll...
tensor(1536)
(1536, 128) (1536, 128) (1536, 128)
[130.00013 130.00013 130.00013 ... 130.00013 130.00013 130.00013] [ True False False ... False False False]
Finished!


In [28]:
tempo.shape

AttributeError: 'list' object has no attribute 'shape'

In [25]:
decode_length

tensor([1536])

In [67]:
print(chord_pred.shape)
print(length1.shape)

torch.Size([1, 272, 96])
torch.Size([1])


In [21]:
length1

tensor([32])

In [81]:
chord_pred_part = chord_pred[0][:val_length[0]].cpu().detach().numpy()
melody_part = melody[0][:val_length[0]]

In [82]:
print(chord_pred_part.shape) 
print(melody_part.shape)

(17, 96)
(17, 576)


In [79]:
che, cc = CHE_and_CC(chord_pred_part, chord_num=96)
ctd = CTD(chord_pred_part, chord_num=96)
ctnctr = CTnCTR(melody_part, chord_pred_part, chord_num=96)
pcs = PCS(melody_part, chord_pred_part, chord_num=96)
mctd = MCTD(melody_part, chord_pred_part, chord_num=96)

In [80]:
ctnctr

0.7381615598885793

In [83]:
weight_chord = np.load('./weight_chord.npy')

In [85]:
weight_chord

array([0.02845915, 0.03385304, 1.72714106, 1.00526172, 0.35642776,
       0.35397114, 0.19579946, 0.40494889, 0.52310953, 1.63496219,
       1.73572526, 1.43952153, 1.48734209, 0.92437248, 1.71022489,
       1.43833576, 0.43730018, 0.18795905, 1.74613962, 0.57533431,
       1.00352852, 1.65197693, 0.26484751, 0.68181945, 0.11128288,
       1.04747427, 1.74613962, 1.62734354, 1.17348092, 0.69099312,
       1.44309059, 1.05443214, 0.65669034, 0.24866699, 1.72203118,
       1.27176957, 1.43243611, 1.67254753, 0.45963138, 0.87833985,
       0.04619661, 0.13839578, 1.73056454, 1.28961567, 0.54841068,
       0.30484281, 0.32264221, 0.42923786, 1.06536889, 1.63496219,
       1.73572526, 0.94436972, 1.67254753, 1.44428422, 1.70023332,
       1.65667895, 0.05119893, 0.20268597, 1.71695144, 1.34525394,
       0.36446245, 1.5276812 , 0.34577022, 0.24873784, 0.06751758,
       1.23402093, 1.72885111, 1.51574619, 1.09407244, 0.23422396,
       1.50399622, 0.73060235, 0.8326846 , 0.0917042 , 1.73400

results/parameter_cvae_sample_result/6-pitch-1-rhythm-1.mid


In [41]:
torch.Tensor([float('1')])

tensor([1.])